In [1]:
# Import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import cv2
import random
from tqdm import tqdm

In [2]:
# Load in dataset
train_dir = "/kaggle/input/70-dog-breedsimage-data-set"
data_df = pd.read_csv(f"{train_dir}/dogs.csv")
data_df.head()

,filepaths,labels,data set
0,train/Afghan/001.jpg,Afghan,train
1,train/Afghan/002.jpg,Afghan,train
2,train/Afghan/003.jpg,Afghan,train
3,train/Afghan/004.jpg,Afghan,train
4,train/Afghan/005.jpg,Afghan,train


In [3]:
# Split into training, validation, and test data
train_df = data_df[data_df.iloc[:, 2] == "train"].copy()
valid_df = data_df[data_df.iloc[:, 2] == "valid"].copy()
test_df = data_df[data_df.iloc[:, 2] == "test"].copy()
train_df.head()

,filepaths,labels,data set
0,train/Afghan/001.jpg,Afghan,train
1,train/Afghan/002.jpg,Afghan,train
2,train/Afghan/003.jpg,Afghan,train
3,train/Afghan/004.jpg,Afghan,train
4,train/Afghan/005.jpg,Afghan,train


In [4]:
# Encode labels
encoded_train_labels, _ = pd.factorize(train_df["labels"])
encoded_valid_labels, _ = pd.factorize(valid_df["labels"])
encoded_test_labels, _ = pd.factorize(test_df["labels"])
train_df["encoded_labels"] = encoded_train_labels
valid_df["encoded_labels"] = encoded_valid_labels
test_df["encoded_labels"] = encoded_test_labels

print(set(encoded_train_labels))
data_df.head()

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69}


,filepaths,labels,data set
0,train/Afghan/001.jpg,Afghan,train
1,train/Afghan/002.jpg,Afghan,train
2,train/Afghan/003.jpg,Afghan,train
3,train/Afghan/004.jpg,Afghan,train
4,train/Afghan/005.jpg,Afghan,train


In [5]:
# Drop unnecessary columns
columns_to_drop = ["labels", "data set"]
train_df = train_df.drop(columns = columns_to_drop)
valid_df = valid_df.drop(columns = columns_to_drop)
test_df = test_df.drop(columns = columns_to_drop)
print(train_df.head())
print(valid_df.head())
print(test_df.head())

              filepaths  encoded_labels
0  train/Afghan/001.jpg               0
1  train/Afghan/002.jpg               0
2  train/Afghan/003.jpg               0
3  train/Afghan/004.jpg               0
4  train/Afghan/005.jpg               0
                filepaths  encoded_labels
8646  valid/Afghan/01.jpg               0
8647  valid/Afghan/02.jpg               0
8648  valid/Afghan/03.jpg               0
8649  valid/Afghan/04.jpg               0
8650  valid/Afghan/05.jpg               0
               filepaths  encoded_labels
7946  test/Afghan/01.jpg               0
7947  test/Afghan/02.jpg               0
7948  test/Afghan/03.jpg               0
7949  test/Afghan/04.jpg               0
7950  test/Afghan/05.jpg               0


In [6]:
# Make labels first column
new_order = ["encoded_labels", "filepaths"]
train_df = train_df[new_order]
valid_df = valid_df[new_order]
test_df = test_df[new_order]
print(train_df.head())
print(valid_df.head())
print(test_df.head())

   encoded_labels             filepaths
0               0  train/Afghan/001.jpg
1               0  train/Afghan/002.jpg
2               0  train/Afghan/003.jpg
3               0  train/Afghan/004.jpg
4               0  train/Afghan/005.jpg
      encoded_labels            filepaths
8646               0  valid/Afghan/01.jpg
8647               0  valid/Afghan/02.jpg
8648               0  valid/Afghan/03.jpg
8649               0  valid/Afghan/04.jpg
8650               0  valid/Afghan/05.jpg
      encoded_labels           filepaths
7946               0  test/Afghan/01.jpg
7947               0  test/Afghan/02.jpg
7948               0  test/Afghan/03.jpg
7949               0  test/Afghan/04.jpg
7950               0  test/Afghan/05.jpg


In [7]:
# TRAINING
# Convert filepaths to numpy arrays
train_df["images"] = train_dir + "/" + train_df["filepaths"]
train_df = train_df.drop(columns=["filepaths"])
train_df["images"] = train_df["images"].apply(lambda path: cv2.imread(path, cv2.IMREAD_GRAYSCALE))
train_df.head()

,encoded_labels,images
0,0,"[[129, 126, 107, 111, 121, 120, 115, 114, 119,..."
1,0,"[[118, 90, 56, 40, 26, 5, 17, 27, 30, 23, 13, ..."
2,0,"[[128, 124, 112, 114, 113, 113, 121, 114, 103,..."
3,0,"[[124, 121, 114, 105, 98, 91, 83, 75, 68, 67, ..."
4,0,"[[184, 187, 189, 187, 183, 181, 183, 187, 189,..."


In [8]:
# Flatten images
pixels = np.stack(train_df['images'].values).reshape(len(train_df), -1)

In [9]:
# Create columns for individual pixels of images
pixels_df = pd.DataFrame(pixels, columns=[f"pixel_{i}" for i in range(pixels.shape[1])])
train_df = pd.concat([train_df['encoded_labels'], pixels_df], axis=1)
train_df.head()

,encoded_labels,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_50166,pixel_50167,pixel_50168,pixel_50169,pixel_50170,pixel_50171,pixel_50172,pixel_50173,pixel_50174,pixel_50175
0,0,129,126,107,111,121,120,115,114,119,...,108,95,89,86,82,79,75,71,69,70
1,0,118,90,56,40,26,5,17,27,30,...,249,249,248,248,247,246,246,246,246,246
2,0,128,124,112,114,113,113,121,114,103,...,139,137,138,140,145,152,156,155,151,148
3,0,124,121,114,105,98,91,83,75,68,...,161,161,161,161,161,162,164,165,166,167
4,0,184,187,189,187,183,181,183,187,189,...,83,79,80,68,39,34,89,163,181,112


In [10]:
# Create numpy array representing all training images
train_data = train_df.to_numpy()
np.random.shuffle(train_data)

In [11]:
# Separate image data from labels
X_train = train_data[:, 1:]
Y_train = train_data[:, 0]

In [12]:
# Normalize data
X_train = X_train / 255

In [13]:
n, m = X_train.shape
output_size = len(set(encoded_train_labels))
hidden_size = m // 4
print(m)
print(hidden_size)
print(output_size)
X_train.shape

50176
12544
70


(7946, 50176)

In [14]:
# Define activation function for hidden layer
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def sigmoid_derivative(Z):
    ds = sigmoid(Z)
    ds = ds * (1 - ds)
    return ds

# Define probability function for output layer
def softmax(Z):
    exps = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    prob = exps / np.sum(exps, axis=1, keepdims=True)
    return prob

In [15]:
#Initialize weights and biases
def init_params():
    W1 = np.sqrt(1. / m) * np.random.randn(m, hidden_size)
    b1 = np.zeros(hidden_size)
    W2 = np.sqrt(1. / hidden_size) * np.random.randn(hidden_size, output_size)
    b2 = np.zeros(output_size)
    
    return W1, b1, W2, b2

In [16]:
# Define forward propagation
def forward_prop(W1, b1, W2, b2, X):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [17]:
# Define backward propagation
def backward_prop(Z1, A1, Z2, A2, W2, X, Y):
    batch_size = len(X)
    dLZ2 = A2.copy()
    dLZ2[range(batch_size), Y] -= 1
    dLZ2 /= batch_size

    dLW2 = np.dot(A1.T, dLZ2)
    dB2 = np.sum(dLZ2, axis=0)

    dLA1 = np.dot(dLZ2, W2.T)
    dLZ1 = dLA1 * sigmoid_derivative(Z1)

    dLW1 = np.dot(X.T, dLZ1)
    dB1 = np.sum(dLZ1, axis=0)

    return dLW1, dB1, dLW2, dB2

In [18]:
def compute_accuracy(X_pred, Y):
    preds = np.argmax(X_pred, axis=1)
    accuracy = np.mean(preds == Y)
    return accuracy

def compute_cross_entropy_loss(X_pred, Y):
    log_likelihood = -np.log(X_pred[:, Y])
    loss = np.sum(log_likelihood) / len(Y)
    return loss

In [19]:
def train(X, Y, learning_rate=0.01, batch_size=32, epochs=5):
    W1, b1, W2, b2 = init_params()
    
    for epoch in range(epochs):
        for batch_start in tqdm(range(0, len(X), batch_size)):
            X_batch = X[batch_start:(batch_start+batch_size), :]
            Y_batch = Y[batch_start:(batch_start+batch_size)]
            
            Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_batch)
            loss = compute_cross_entropy_loss(A2, Y_batch)
            accuracy = compute_accuracy(A2, Y_batch)
            dLW1, dB1, dLW2, dB2 = backward_prop(Z1, A1, Z2, A2, W2, X_batch, Y_batch)

            # Update weights and biases
            W1 -= learning_rate * dLW1
            b1 -= learning_rate * dB1
            W2 -= learning_rate * dLW2
            b2 -= learning_rate * dB2

            if (batch_start // batch_size) % 10 == 0:
                print(f"Batch {batch_start // batch_size}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")
                
    return W1, b1, W2, b2

In [20]:
W1, b1, W2, b2 = train(X_train, Y_train, 0.0001, 64, 10)

  1%|          | 1/125 [00:07<14:49,  7.17s/it]

Batch 0: Loss = 286.0406, Accuracy = 0.0000


  9%|▉         | 11/125 [01:19<13:34,  7.15s/it]

Batch 10: Loss = 284.4180, Accuracy = 0.0000


 17%|█▋        | 21/125 [02:30<12:22,  7.14s/it]

...

 89%|████████▉ | 111/125 [13:26<01:39,  7.10s/it]

Batch 110: Loss = 274.6432, Accuracy = 0.0312


 97%|█████████▋| 121/125 [14:39<00:28,  7.15s/it]

Batch 120: Loss = 272.0798, Accuracy = 0.0469


100%|██████████| 125/125 [15:07<00:00,  7.26s/it]
